In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import tensorflow_hub as hub
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import numpy as np
import nltk
import re

In [3]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv",names = ['target','id','date','flag','user','tweet'], encoding="ISO-8859-1")
del data['id']
del data['date']
del data['flag']
del data['user']

data = shuffle(data)
data.reset_index(inplace=True, drop=True)

data.head()

,target,tweet
0,0,@titiburgos TITI VEO NEGROOOO!! I feel super ...
1,4,@MollieBrislin Wow...that's crazy! I can hardl...
2,4,"Word to all those i call mom, and there are a ..."
3,4,"@PoisonTheMonkey LOL Maybe. Oh well, maybe he..."
4,0,never gets on twitter anymore


In [5]:
X = []
for sen in data['tweet']:
  X.append(sen)
Y = np.array(data['target']/4)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [8]:
def preprocess(text):
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
          tokens.append(token)
    return " ".join(tokens)

def encoding(tweet):
  tweet = preprocess(tweet)
  tokens = tweet.split(" ")
  embedding = []
  for token in tokens:
    embedding.append(embed([token]).numpy()[0])

  embedding = np.array(embedding)
  padded_tweet = padding(embedding)
  return padded_tweet

def padding(encoded_tweet):
  count = 70 - encoded_tweet.shape[0]
  pad = np.zeros((1,250))
  for i in range(count):
    encoded_tweet = np.concatenate((pad,encoded_tweet), axis=0)
  return encoded_tweet


def processed_tweets(tweets):
  x = []
  for tweet in tweets:
    enc = encoding(tweet)
    x.append(enc)
  return np.array(x)

In [ ]:
X_train = processed_tweets(X_train)
X_test = processed_tweets(X_test)

In [9]:
tweet = ['Pained by the loss of lives due to a fire at a hospital in Bharuch. Condolences to the bereaved families.','To add further momentum to our strong strategic partnership, President Putin and I have agreed to establish a Ministerial Dialogue between our Foreign and Defence Ministers.']
x = processed_tweets(tweet)
y = np.array([0,1])

In [10]:
clf = Sequential()

clf.add(LSTM(128, activation='relu', return_sequences=True))
clf.add(Dropout(0.2))

clf.add(LSTM(128, activation='relu'))
clf.add(Dropout(0.2))

clf.add(Dense(32, activation='relu'))
clf.add(Dropout(0.2))

clf.add(Dense(1,activation='sigmoid'))

clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
clf.fit(x, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 52ms/step - loss: 0.6288 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 52ms/step - loss: 0.6019 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 52ms/step - loss: 0.5757 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 47ms/step - loss: 0.5363 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 51ms/step - loss: 0.4703 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 52ms/step - loss: 0.4311 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 50ms/step - loss: 0.3097 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 49ms/step - loss: 0.3018 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 48ms/step - loss: 0.2798 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 52ms/step - loss: 0.1587 - accuracy: 1.0000


In [14]:
clf.predict(processed_tweets(tweet))

array([[0.06067079],
       [1.        ]], dtype=float32)